In [1]:
import pandas as pd
import numpy as np
import json
from buckaroo.buckaroo_widget import BuckarooWidget, disable, enable, df_to_obj
import matplotlib

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
from buckaroo import  summary_stats 
df4 = pd.read_csv('WPP2022_Demographic_Indicators_Medium.csv')

/var/folders/th/jfr8ccjx09g2j_18ty_t3xnh0000gn/T/ipykernel_11949/3496868452.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv('WPP2022_Demographic_Indicators_Medium.csv')


In [3]:
df6 =  pd.read_parquet('1993-flights.gzip.parquet')
len(df6)

5070501

In [4]:
df6

BuckarooWidget(commandConfig={'argspecs': {'dropcol': [None], 'to_datetime': [None], 'safeint': [None], 'filln…

In [ ]:
%timeit summary_stats.DfStats(summary_stats.sample(df6, 5000))

In [ ]:
len(df6)

In [ ]:
len(df6.columns)

In [ ]:
%timeit summary_stats.DfStats(df6.sample(50000))

In [ ]:
NUM = 50_000
print(NUM * len(df6.columns) // 1_000_000)
print(NUM * len(df6.columns) )

%timeit summary_stats.DfStats(df6.sample(NUM))

In [ ]:
%prun df6.sample(100000)

In [ ]:
%timeit summary_stats.DfStats(df6[:500_000])

In [ ]:
%prun -s cumtime -l 100 summary_stats.DfStats(df6[:500_000])

In [ ]:
%timeit summary_stats.DfStats(df6[:1000_000])

In [ ]:
df4

In [ ]:
df3 = pd.read_parquet('Covid-data.gzip.parquet')
df3

In [ ]:
df3[:5000]

In [ ]:
table_sumarize(df3[:500_00])

In [ ]:
df = df3[:2]

In [ ]:
json.loads(df.to_json(orient='table', indent=2, default_handler=str))

In [ ]:
df.to_dict('records')

In [ ]:
help(df.to_dict)

In [ ]:
from buckaroo.buckaroo_widget import df_to_obj as df_to_obj_orig

In [ ]:
%timeit  len(summary_stats.sample(df3, 5000))

In [ ]:
def get_outlier_idxs(ser):
    if not pd.api.types.is_numeric_dtype(ser.dtype):
        return []
    outlier_idxs = []
    outlier_idxs.extend(ser.nlargest(5).index)
    outlier_idxs.extend(ser.nsmallest(5).index)
    return outlier_idxs

def sample(df, sample_size=500, include_outliers=True):
    if len(df) <= sample_size:
        return df
    sdf = df.sample(np.min([sample_size, len(df)]))
    if include_outliers:
        outlier_idxs = []
        for col in df.columns:
            outlier_idxs.extend(get_outlier_idxs(df[col]) )
        outlier_idxs.extend(sdf.index)
        uniq_idx = np.unique(outlier_idxs)
        return df.iloc[uniq_idx]
    return sdf
%timeit len(sample(df3, 5000))

In [ ]:
df_s = sample(df3, 5000)
df_s

In [ ]:
%prun -s cumtime -l 100 len(sample(df3, 5000))

In [ ]:
%prun -s cumtime -l 100 len(summary_stats.sample(df3, 5000))

In [ ]:
%prun -s cumtime -l 100 len(df_to_obj_orig( df3[:100_000]))

In [ ]:
from pandas.io.json import dumps as pdumps
from buckaroo.summary_stats import table_sumarize
def df_to_obj(df, order = None):
    if order is None:
        order = df.columns
    #obj = json.loads(df.to_json(orient='table', indent=2, default_handler=str))
    fields=[{'name':'index'}]
    for c in order:
        fields.append({'name':c})
    obj = {}
    obj['data'] = df.to_dict('records')
    #obj['table_hints'] = json.loads(pdumps(table_sumarize(df)))

    obj['schema'] = dict(fields=fields)
    return obj
%prun -s cumtime -l 100 len(df_to_obj(  df3[:500_000]))

In [ ]:
%prun -l 100 -s cumtime bw = BuckarooWidget(df3)

In [ ]:
%timeit bw = BuckarooWidget(df3[:50000])

In [ ]:
df3 = pd.read_parquet('Covid-data.gzip.parquet')
%timeit bw = BuckarooWidget(df3)

In [ ]:
%timeit bw = BuckarooWidget(df3[:], really_reorder_columns=True )

In [ ]:
%

In [ ]:
df4

In [ ]:
dfs = summary_stats.DfStats(df4)
sdf = dfs.sdf

In [ ]:
disable()

In [ ]:
sdf[sdf.columns[(sdf.loc['distinct_per'] < .3) | (sdf.loc['distinct_count'] < 50)]]

In [ ]:
sdf.columns[sdf.loc['distinct_per'] < .3 | sdf.loc['distinct_count'] < 50]

In [ ]:
sdf.columns[sdf.loc['unique_per'] < .3]

In [ ]:
ab = sdf.loc['distinct_count'] > 1
len(sdf.columns), len(sdf.columns[ab])

In [ ]:
%timeit summary_stats.DfStats(df4)

In [ ]:
is3 = df4['ISO3_code']
is3

In [ ]:
make_num_categorical(is3)

In [ ]:
make_num_categorical(is3.drop_duplicates())

In [ ]:
def make_num_categorical(ser):
    ser_uniq = ser.dropna().unique()
    name_to_idx = {name:idx for idx, name in enumerate(ser_uniq)}
    #needs to be vectorized
    num_categorical = ser.dropna().apply(lambda x:name_to_idx[x])
    return num_categorical


In [ ]:
ab = is3.astype('category')
ab.cat.codes

In [ ]:
print(summary_stats.get_cor_pair_dict(df4, summary_stats.summarize_df(df4)))

In [ ]:
get_cor_pair_dict(df4, summary_stats.summarize_df(df4))

In [ ]:
cols = [make_num_categorical(df4['Births']), make_num_categorical(df4['ISO2_code']), 
                        make_num_categorical(df4['ISO3_code']), make_num_categorical(df4['LBsurvivingAge1'])]

In [ ]:
disable()

In [ ]:
pd.DataFrame([make_num_categorical(df4['Births']), make_num_categorical(df4['ISO2_code']), 
                        make_num_categorical(df4['ISO3_code']), make_num_categorical(df4['LBsurvivingAge1'])])

In [ ]:
corr_df = pd.DataFrame(
    {'Births':make_num_categorical(df4['Births']), 'ISO2_code':make_num_categorical(df4['ISO2_code']), 
      'ISO3_code':make_num_categorical(df4['ISO3_code']), 'LBsurvivingAge1': make_num_categorical(df4['LBsurvivingAge1'])})
corr_df.corr()

In [ ]:
corr_df = pd.DataFrame(
    {'Births':summary_stats.make_num_categorical(df4['Births']), 'ISO2_code': summary_stats.make_num_categorical(df4['ISO2_code']), 
      'ISO3_code':summary_stats.make_num_categorical(df4['ISO3_code']), 'LBsurvivingAge1': summary_stats.make_num_categorical(df4['LBsurvivingAge1'])})
corr_df.corr()

In [ ]:
corr_df

In [ ]:
enable()

In [ ]:
pd.DataFrame({'raw':df4['ISO3_code'], 'old_cat': summary_stats.make_num_categorical(df4['ISO3_code']), 
              #'cat':df4['ISO3_code'].astype('category').cat, 
              'num': df4['ISO3_code'].astype('category').cat.codes}).dropna().drop_duplicates()

In [ ]:
def make_num_categorical(ser):
    return ser.dropna().astype('category').cat.codes

def get_cor_pair_dict(df, sdf):
    #we need to remove columns with only nans or a single value, they mess up corr()

    #this needs to be vectorized
    corrable_cols = sdf.columns[
        (sdf.loc['distinct_count'] > 1) &
        ((sdf.loc['distinct_per'] < .3) |
         (sdf.loc['distinct_count'] < 50))]
    #corrable_cols = sdf.columns[sdf.loc['distinct_count'] > 1]
    num_df =  pd.DataFrame({col:make_num_categorical(df[col]) for col in corrable_cols})

    corr_df = num_df.corr()
    high_corr_df = corr_df[corr_df > 0.6]
    
    print(high_corr_df)

    cor_dict = {}
    for col in high_corr_df.columns:
        #columns with high correlation that aren't the column itself
        other_cor_cols = high_corr_df[col].dropna().drop(col)
        cor_cols = other_cor_cols.index.values
        if len(cor_cols) > 0:
            cor_dict[col] = cor_cols.tolist()
    return cor_dict

%prun -s cumtime print(get_cor_pair_dict(df4, summary_stats.summarize_df(df4)))

In [ ]:
%prun -s cumtime get_cor_pair_dict(df4, summary_stats.summarize_df(df4))

In [ ]:
%prun -s cumtime summary_stats.DfStats(df4)

In [ ]:
bw = BuckarooWidget(df4[:5000])
bw

In [ ]:
from buckaroo.summary_stats import order_groupings, find_groupings
def order_columns(sdf, corr_pair_dict):
    grouping_col_scores = sdf.loc[['grouping_score_integer', 'grouping_score_numeric']].sum()
    duplicate_col_rankings = grouping_col_scores.sort_values().index[::-1].values

    groupings = find_groupings(corr_pair_dict)
    #print("groupings", groupings)
    first_cols, duplicate_cols = order_groupings(groupings, duplicate_col_rankings)
    
    #print("duplicate_cols", duplicate_cols)
    print("first_cols", first_cols)
    sdf.loc['first_col':, list(set(first_cols))] = 5
    print(type(sdf['ISO3_code']))

    sdf.loc['is_duplicate':, duplicate_cols] = -5
    print(sdf.index)
    col_scores = sdf.loc[['one_distinct', 'first_col', 'datetime_score', 'is_duplicate']].sum()
    return col_scores.sort_values().index.values[::-1]
stats = summary_stats.DfStats(df4[:5000], order_col_func=order_columns)
type(stats.sdf['ISO3_code'])

In [ ]:
from buckaroo.summary_stats import set_when
def add_col_rankings(df, sdf):
    sdf.loc['one_distinct'] = 0

    only_ones = (sdf.loc['distinct_count'] <= 1)
    sdf.loc['one_distinct', only_ones[only_ones==True].index.values] = -20
    
    sdf.loc['first_col'] = 0
    sdf.loc['is_duplicate'] = 0
    set_when(sdf, 'is_datetime', 'datetime_score', 11, 0)
    
    set_when(sdf, 'is_integer', 'grouping_score_integer', -3, 0)
    set_when(sdf, 'is_numeric', 'grouping_score_numeric', -3, 5)
stats = summary_stats.DfStats(df4[:5000], [add_col_rankings])
type(stats.sdf['ISO3_code'])

In [ ]:
BuckarooWidget.summary_df_cols

In [ ]:
ab = summary_stats.summarize_df(df4)

In [ ]:
ab['ISO3_code']

In [ ]:
df4['ISO3_code']

In [ ]:
stats.sdf.loc[BuckarooWidget.summary_df_cols]['ISO3_code'].dtype

In [ ]:
disable()

In [ ]:
ssdf['ISO3_code']

In [ ]:
stats.sdf['ISO3_code']

In [ ]:
for col in ssdf.columns:
    print(col)
    summary_stats.table_sumarize_ser(ssdf[col])
#summary_stats.table_sumarize(ssdf)

In [ ]:
#summary_stats.table_sumarize(df4)

In [ ]:
stats = summary_stats.DfStats(ssdf)

In [ ]:
disable()
stats.sdf

In [ ]:
BuckarooWidget.summary_df_cols

In [ ]:
ssdf = stats.sdf.loc[BuckarooWidget.summary_df_cols]


In [ ]:
ssdf.dtypes.tail(50)

In [ ]:
#df9 = pd.read_csv('scorecard.csv')
bw = BuckarooWidget(df4[:5000])
bw

In [ ]:
bw

In [ ]:
BuckarooWidget.summary_df_cols

In [ ]:
summary_stats.DfStats(df4)

In [ ]:

class DfStats(object):
    def __init__(self,
            df,
            annotate_funcs=[],
            summary_func=summary_stats.summarize_df,
            order_col_func=summary_stats.order_columns):
            #summary_func=summarize_df,
            #order_col_func=order_columns):

        self.df = df
        self.sdf = summary_func(df)
        for func in annotate_funcs:
            func(df, self.sdf)
        self.cpd = summary_stats.get_cor_pair_dict(self.df, self.sdf)
        self.col_order = order_col_func(self.sdf, self.cpd)

In [ ]:
def add_col_rankings(df, sdf):
    sdf.loc['one_distinct'] = 0

    only_ones = (sdf.loc['distinct_count'] <= 1)
    sdf.loc['one_distinct', only_ones[only_ones==True].index.values] = -20
    
    sdf.loc['first_col'] = 0
    sdf.loc['is_duplicate'] = 0
    set_when(sdf, 'is_datetime', 'datetime_score', 11, 0)
    
    set_when(sdf, 'is_integer', 'grouping_score_integer', -3, 0)
    set_when(sdf, 'is_numeric', 'grouping_score_numeric', -3, 5)


In [ ]:
from buckaroo.summary_stats import set_when, find_groupings, order_groupings

def find_groupings(corr_pairs):
    all_groupings = []
    for key, other_key_list in corr_pairs.items():
        ab = other_key_list.copy()
        ab.append(key)
        all_groupings.append(set(ab))
    return np.unique(all_groupings)

def order_groupings(grps, ranked_cols):
    first_cols, rest_cols = [], []
    for col in ranked_cols:
        for grp in grps:
            if col in grp:
                first_cols.append(col)
                rest_cols.extend(list(summary_stats.without(grp, [col])))
                grps = summary_stats.without(grps, [grp])
    return first_cols, list(set(rest_cols))

def order_columns(sdf, corr_pair_dict):
    grouping_col_scores = sdf.loc[['grouping_score_integer', 'grouping_score_numeric']].sum()
    duplicate_col_rankings = grouping_col_scores.sort_values().index[::-1].values

    groupings = find_groupings(corr_pair_dict)
    print("groupings", groupings)
    first_cols, duplicate_cols = order_groupings(groupings, duplicate_col_rankings)
    
    print("duplicate_cols", duplicate_cols)
    sdf.loc['first_col':, first_cols] = 5
    sdf.loc['is_duplicate':, duplicate_cols] = -5
    print(sdf.index)
    col_scores = sdf.loc[['one_distinct', 'first_col', 'datetime_score', 'is_duplicate']].sum()
    return col_scores.sort_values().index.values[::-1]

In [ ]:
df4.columns[:25]

In [ ]:
df_prob = df4[['SortOrder',
               'LocID', 'Notes', 'ISO3_code', 'ISO2_code', 'SDMX_code',
       'LocTypeID', 'LocTypeName', 'ParentID', 
               'Location', 'VarID', 'Variant',
       'Time', 'TPopulation1Jan',
               'TPopulation1July', 'TPopulationMale1July',
       'TPopulationFemale1July', 
               'PopDensity', 'PopSexRatio', 
               'MedianAgePop',
       'NatChange', 'NatChangeRT', 'PopChange',# 'PopGrowthRate',
       #'DoublingTime'
              ]][:300]
DfStats(df_prob, [add_col_rankings], order_col_func=order_columns)

In [ ]:
DfStats(df4[df4.columns[:]][:300], [add_col_rankings], order_col_func=order_columns)

In [ ]:
#bw = BuckarooWidget(df9, sampled=False)
#bw

In [ ]:
disable()

In [ ]:
df_to_obj(bw.df)['schema']

In [ ]:
df_to_obj(bw.df, bw.stats.col_order)['schema']

In [ ]:
bw.df

In [ ]:
bw.stats.col_order

In [ ]:
from pandas.io.json import dumps as pdumps
from buckaroo.summary_stats import table_sumarize
def df_to_obj(df, order = None):
    if order is None:
        order = df.columns
    obj = json.loads(df.to_json(orient='table', indent=2, default_handler=str))
    obj['table_hints'] = json.loads(pdumps(table_sumarize(df)))
    obj['schema'] = dict(
            fields=[{'name':c} for c in order])
    return obj
df_to_obj(df9[:3], ['year', 'inst_name'])

In [ ]:
class DfStats(object):
    def __init__(self,
            df,
            summary_func=summary_stats.summarize_df,
            order_col_func=summary_stats.order_columns):
        self.df = df
        self.sdf = summary_func(df)
        self.cpd = summary_stats.get_cor_pair_dict(self.df, self.sdf)
        self.col_order = order_col_func(self.sdf, self.cpd)

In [ ]:
ab = df9['count_not_working']
ab.nlargest(5).index

In [ ]:
df9['count_not_working']

In [ ]:
from buckaroo.summary_stats import set_when, find_groupings, order_groupings
def order_columns(summary_stats_df, corr_pair_dict):
    sdf = summary_stats_df.copy()
    sdf.loc['one_distinct'] = 0

    only_ones = (sdf.loc['distinct_count'] <= 1)
    sdf.loc['one_distinct', only_ones[only_ones==True].index.values] = -20
    
    sdf.loc['first_col'] = 0
    sdf.loc['is_duplicate'] = 0
    set_when(sdf, 'is_datetime', 'datetime_score', 11, 0)
    
    set_when(sdf, 'is_integer', 'grouping_score_integer', -3, 0)
    set_when(sdf, 'is_numeric', 'grouping_score_numeric', -3, 5)
    grouping_col_scores = sdf.loc[['grouping_score_integer', 'grouping_score_numeric']].sum()
    duplicate_col_rankings = grouping_col_scores.sort_values().index[::-1].values

    groupings = find_groupings(corr_pair_dict)
    first_cols, duplicate_cols = order_groupings(groupings, duplicate_col_rankings)
    
    sdf.loc['first_col':, first_cols] = 5
    sdf.loc['is_duplicate':, duplicate_cols] = -5
    print(sdf.index)
    col_scores = sdf.loc[['one_distinct', 'first_col', 'datetime_score', 'is_duplicate']].sum()
    return col_scores.sort_values().index.values[::-1]


In [ ]:

def reorder_columns(df):
    tdf_stats = summary_stats.summarize_df(df)
    cpd = summary_stats.get_cor_pair_dict(df, tdf_stats)
    col_order = order_columns(tdf_stats, cpd)
    #return 5
    return df[col_order]
    #except Exception as e:
    #    print("error reordering columns", e)
    #    return df
bc = reorder_columns(df9)

In [ ]:
dfs = DfStats(df9, order_col_func=order_columns)
dfs.col_order

In [ ]:
len(np.unique(dfs.col_order)), len(dfs.col_order)

In [ ]:
df9[dfs.col_order]

In [ ]:
ab = BuckarooWidget(df9)
ab.dfConfig

In [ ]:
ab.summary_df

In [ ]:
from buckaroo.summary_stats import summarize_column

In [ ]:
df6.columns

In [ ]:
from buckaroo import summary_stats
def summarize_numeric2(ser):

    num_stats = dict(
        min=ser.min(),
        max=ser.max(),
        mean=ser.mean())
    #num_stats.update(summary_stats.summarize_string(ser))
    return num_stats


In [ ]:
ab = df6['DepTime']
%timeit ab.describe()

In [ ]:
%timeit summarize_numeric2(df6['DepTime'])

In [ ]:
%timeit summarize_column(df6['DepTime'])

In [ ]:
ab.nsmallest(5)

In [ ]:
def get_outlier_idxs(ser):
    outlier_idxs = []
    try:
        highest = ser.nlargest(5)
        smallest = ser.nlargest(5)
    except Exception as e:
        return []
        #print(e)
        #idxs = ser.index
    outlier_idxs.extend(smallest.index)
    outlier_idxs.extend(highest.index)
    return outlier_idxs

def sample2(df, sample_size=500, include_outliers=True):
    if len(df) <= sample_size:
        return df
    sdf = df.sample(np.min([sample_size, len(df)]))
    if include_outliers:
        outlier_idxs = []
        for col in df.columns:
            outlier_idxs.extend(get_outlier_idxs(df[col]) )
        outlier_idxs.extend(sdf.index)
        uniq_idx = np.unique(outlier_idxs)
        return df.iloc[uniq_idx]
    return sdf


In [ ]:
%timeit sample2(df6[['DepTime']])

In [ ]:
%timeit buckaroo_widget.sample(df6[['DepTime']])

In [ ]:
from buckaroo import buckaroo_widget

In [ ]:
df6 =  pd.read_parquet('1993-flights.gzip.parquet')
df6